In [8]:
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

# You can also import other things ...
# YOUR CODE HERE (OPTION)
from sklearn.preprocessing import FunctionTransformer

DO KHỐI LƯỢNG DATA KHỔNG LỒ (2.5GB), NHÓM CHỈ THỰC HIỆN KHÁM PHÁ VỚI 12 FILE TRƯỚC, SAU ĐÓ SẼ THIẾT KẾ ĐOẠN CODE ĐỂ KHÁM PHÁ TỰ ĐỘNG CÁC FILE CÒN LẠI ĐỂ ĐƯA RA THÔNG KẾ SAU CÙNG

In [16]:
df0=pd.read_csv('scraped_data/crawling_0.csv')
df1=pd.read_csv('scraped_data/crawling_1.csv')
df2=pd.read_csv('scraped_data/crawling_2.csv')
df3=pd.read_csv('scraped_data/crawling_3.csv')
df4=pd.read_csv('scraped_data/crawling_4.csv')
df5=pd.read_csv('scraped_data/crawling_5.csv')
df6=pd.read_csv('scraped_data/crawling_6.csv')
df7=pd.read_csv('scraped_data/crawling_7.csv')
df8=pd.read_csv('scraped_data/crawling_8.csv')
df9=pd.read_csv('scraped_data/crawling_9.csv')
df10=pd.read_csv('scraped_data/crawling_10.csv')
df11=pd.read_csv('scraped_data/crawling_11.csv')
data_df=pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])

In [17]:
data_df.reset_index(drop=True, inplace=True)

# Khám phá dữ liệu

In [18]:
data_df.head()

,links,title,description,content,class
0,https://tuoitre.vn/tong-thong-trump-xac-nhan-k...,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,https://tuoitre.vn/dat-nuoc-dat-niem-tin-vao-n...,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,https://tuoitre.vn/luat-su-my-phan-bien-ong-tr...,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,https://tuoitre.vn/thanh-pho-phu-quoc-se-phat-...,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,https://tuoitre.vn/ong-trump-nguoi-ung-ho-toi-...,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [19]:
data_df.shape

(52000, 5)

In [20]:
# Các cột hiện có kiểu dữ liệu gì?
data_df.dtypes

links          object
title          object
description    object
content        object
class          object
dtype: object

### Kiểm tra và xử lí dữ liệu trùng

In [21]:
data_df[data_df.duplicated(keep=False)].sort_values(by=['links'])

,links,title,description,content,class
48880,/dang-sai-lech-vu-dong-tam-facebooker-chuong-m...,"Đăng sai lệch vụ Đồng Tâm, Facebooker ‘Chương ...","TTO - Ngày 20-1, Cơ quan cảnh sát điều tra Côn...","Facebooker ""Chương May Mắn"" tên thật là Chung ...",Pháp luật
48879,/dang-sai-lech-vu-dong-tam-facebooker-chuong-m...,"Đăng sai lệch vụ Đồng Tâm, Facebooker ‘Chương ...","TTO - Ngày 20-1, Cơ quan cảnh sát điều tra Côn...","Facebooker ""Chương May Mắn"" tên thật là Chung ...",Pháp luật
307,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",TTO - Sau nhiều ngày số mắc mới khá cao và dồn...,"Theo Bộ Y tế, 1 ca mắc mới ngày 6-1 là ca nhập...",Sức khỏe
254,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",TTO - Sau nhiều ngày số mắc mới khá cao và dồn...,"Theo Bộ Y tế, 1 ca mắc mới ngày 6-1 là ca nhập...",Sức khỏe
179,https://tuoitre.vn/affordable-luxury-dong-san-...,'Affordable luxury' - dòng sản phẩm đột phá về...,Apec Mandala Wyndham Mũi Né đánh dấu bước đầu ...,Apec Group và thương hiệu bất động sản mang tí...,Cần biết
...,...,...,...,...,...
229,https://tuoitre.vn/vuong-quoc-anh-ghi-nhan-hon...,Vương quốc Anh ghi nhận hơn 1.000 người chết m...,TTO - Vương quốc Anh ngày 6-1 ghi nhận thêm 1....,"Theo hãng tin Reuters, trong ngày 6-1, Vương q...",Sức khỏe
304,https://tuoitre.vn/wall-street-journal-bac-kin...,Wall Street Journal: Bắc Kinh ép Jack Ma chia ...,TTO - Chính quyền Trung Quốc đang cố gắng buộc...,Báo Wall Street Journal nhận định Jack Ma có r...,Kinh doanh
251,https://tuoitre.vn/wall-street-journal-bac-kin...,Wall Street Journal: Bắc Kinh ép Jack Ma chia ...,TTO - Chính quyền Trung Quốc đang cố gắng buộc...,Báo Wall Street Journal nhận định Jack Ma có r...,Kinh doanh
286,https://tuoitre.vn/xu-phat-3-nguoi-75-trieu-do...,Xử phạt 3 người 75 triệu đồng vì khai thác kho...,TTO - Chủ đất và hai người tham gia lấy phần c...,"Ngày 6-1, Công an tỉnh Trà Vinh tống đạt các q...",Pháp luật


In [22]:
data_df.drop_duplicates(inplace=True)


### Kiểm tra dữ liệu thiếu?

In [23]:
data_df.fillna('',inplace=True)

### Kiểm tra số giá trị thiếu ở từng cột

In [24]:
(data_df['links']=='').sum()

0

In [25]:
(data_df['title']=='').sum()

244

In [26]:
(data_df['description']=='').sum()

290

In [27]:
(data_df['content']=='').sum()

904

In [28]:
(data_df['class']=='').sum()

0

In [29]:
data_df[(data_df['title']=='')|
        (data_df['description']=='')|
        (data_df['content']=='')]

,links,title,description,content,class
29,https://tuoitre.vn/lang-kinh-24g-xe-du-ben-coc...,"Lăng kính 24h: 'Xe dù, bến cóc' - vấn nạn nhiề...",,,Cần biết
123,https://tuoitre.vn/doc-bao-cung-ban-8-1-viet-n...,Đọc báo cùng bạn 8-1: Việt Nam sẽ có vắc xin d...,TTO - Chương trình hôm nay có nhiều thông tin ...,,Bạn đọc làm báo
129,https://tuoitre.vn/video-daisuke-matsui-the-hi...,Video Daisuke Matsui thể hiện đẳng cấp ngay lầ...,,,Thể thao
137,https://tuoitre.vn/duoc-hlv-huynh-duc-kem-chat...,"Được HLV Huỳnh Đức 'kèm chặt', Đức Chinh liền ...","TTO - Chiều 7-1, tiền đạo Hà Đức Chinh đã gây ...",,Thể thao
166,https://tuoitre.vn/lang-kinh-24g-doi-no-thue-h...,Lăng kính 24g: Đòi nợ thuê 'hết đất' lộng hành,,,Cần biết
...,...,...,...,...,...
51672,/tan-chu-tich-clb-sai-gon-chung-toi-se-lam-cho...,,,,Tin tức
51691,/toan-canh-muc-an-cua-hai-cuu-bo-truong-va-cac...,Toàn cảnh mức án của hai cựu bộ trưởng và các ...,"TTO - Sau khoảng 2 tuần xét xử, trưa 28-12, HĐ...",,Pháp luật
51717,/10-nam-van-dong-duoc-334-ti-dong-cho-quy-vi-b...,,,,Thời sự
51828,/video-bong-vao-luoi-ca-met-trong-tai-van-tu-c...,"Video: Bóng vào lưới cả mét, trọng tài vẫn từ ...",TTO - Trong trận đấu giữa Cosenza Calcio và Em...,,Thể thao


In [30]:
list(data_df[(data_df['title']=='')|
        (data_df['description']=='')|
        (data_df['content']=='')]['links'])

['https://tuoitre.vn/lang-kinh-24g-xe-du-ben-coc-van-nan-nhieu-nam-chua-the-xu-ly-20210108170033508.htm',
 'https://tuoitre.vn/doc-bao-cung-ban-8-1-viet-nam-se-co-vac-xin-dich-vu-cuoi-quy-1-20210108035639378.htm',
 'https://tuoitre.vn/video-daisuke-matsui-the-hien-dang-cap-ngay-lan-dau-ra-san-cho-clb-sai-gon-2021010804083996.htm',
 'https://tuoitre.vn/duoc-hlv-huynh-duc-kem-chat-duc-chinh-lien-lap-cu-dup-ban-thang-2021010720140594.htm',
 'https://tuoitre.vn/lang-kinh-24g-doi-no-thue-het-dat-long-hanh-20210107165553962.htm',
 'https://tuoitre.vn/doc-bao-cung-ban-7-1-co-mat-bang-cua-ngo-tp-hcm-se-thong-20210107041038267.htm',
 'https://tuoitre.vn/iran-pho-dien-may-bay-khong-nguoi-lai-mang-ten-lua-20210107043956282.htm',
 'https://tuoitre.vn/khoanh-khac-cac-nghi-si-cong-hoa-vo-tay-thach-thuc-kiem-phieu-bang-arizona-20210107033815031.htm',
 'https://tuoitre.vn/lang-kinh-24g-canh-giac-voi-toi-pham-trom-cap-xe-may-20210106170505071.htm',
 'https://tuoitre.vn/doc-bao-cung-ban-6-1-thit-nhap-de

### Sau khi kiểm tra các link thiếu dữ liệu là các trang tin tức qua video nên ta có thể drop hết các giá trị trên

In [31]:
data_df=data_df[~((data_df['title']=='')|
                  (data_df['description']=='')|
                  (data_df['content']==''))]

In [32]:
data_df

,links,title,description,content,class
0,https://tuoitre.vn/tong-thong-trump-xac-nhan-k...,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,https://tuoitre.vn/dat-nuoc-dat-niem-tin-vao-n...,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,https://tuoitre.vn/luat-su-my-phan-bien-ong-tr...,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,https://tuoitre.vn/thanh-pho-phu-quoc-se-phat-...,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,https://tuoitre.vn/ong-trump-nguoi-ung-ho-toi-...,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới
...,...,...,...,...,...
51995,/tham-sat-o-thai-nguyen-nghi-pham-thua-nhan-nh...,Thảm sát ở Thái Nguyên: Nghi phạm thừa nhận nh...,"TTO - Tại cơ quan công an, bước đầu nghi phạm ...",Sau khi truy bắt Hoàng Văn Chín - nghi phạm gâ...,Pháp luật
51996,/mobifone-dat-loi-nhuan-hon-6-000-ty-dong-tron...,MobiFone đạt lợi nhuận hơn 6.000 tỉ đồng trong...,TTO – Mức lợi nhuận trước thuế của MobiFone ướ...,"Ngày 26-12, MobiFone công bố kết thúc năm 2019...",Kinh doanh
51997,/chuan-bi-co-le-hoi-hoa-anh-dao-tai-tp-hcm-201...,Chuẩn bị có lễ hội hoa anh đào tại TP.HCM,TTO - Làm việc với đoàn giám sát của HĐND TP.H...,"Ngày 26-12, Đoàn công tác của HĐND TP.HCM đã c...",Du Lịch
51998,/xet-xu-vu-nu-sinh-giao-ga-bui-van-cong-lien-t...,Xét xử vụ nữ sinh giao gà: Bùi Văn Công liên t...,"TTO - Được gọi lên xét hỏi, bị cáo Bùi Văn Côn...","Chiều 26-12, phiên tòa xét xử 9 bị cáo bắt cóc...",Pháp luật


### Thuộc tính `links` chỉ là giá trị định danh nên sẽ drop đi

In [33]:
data_df.drop(['links'],axis=1,inplace=True)

### Xét trong thuộc tính `description` thì từ `TTO`(Tuổi Trẻ Online) xuất hiện thường xuyên nhưng không mang ý nghĩa đáng kể nên ta bỏ nó đi

In [34]:
data_df['description']=data_df['description'].str.replace("TTO - ",'')
data_df['description']=data_df['description'].str.replace("TTO – ",'')

## Tiền xử lí (tách các tập)

In [35]:
# Tách X và y
y_sr = data_df["class"] # sr là viết tắt của series
X_df = data_df.drop("class", axis=1)

In [36]:
# Tách tập huấn luyện và tập validation theo tỉ lệ 70%:30%

#train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3,stratify=y_sr, random_state=0)
y_sr.value_counts()[:50]


Thời sự                8484
Thế giới               6142
Kinh doanh             4554
Pháp luật              4426
Sức khỏe               4211
Thể thao               4182
Cần biết               2871
Giáo dục               2578
Văn hóa                1563
Nhịp sống trẻ          1561
Giải trí               1412
Khoa học               1390
Du Lịch                1310
Bạn đọc làm báo         989
Công nghệ               848
Doanh nghiệp            698
Phóng sự                693
Nhà đất                 650
Giả - Thật              501
Tuyển sinh              387
Bình luận               372
Tài chính               159
Mua sắm                 143
Xe                      135
Xã hội                   92
Nhịp sống số             90
Bút Bi                   58
Học đường                56
Tin tức                  52
Biết để khỏe             47
Đời sống                 41
Dự án                    37
Góc học tập              31
Sách                     28
Media                    26
Cơ hội du lịch      

In [37]:
y_sr.value_counts()[49:]

Muôn màu            4
Việc làm            4
Xu hướng            4
Cộng đồng           4
Dinh dưỡng          4
TV Show             3
Pháp lý             3
Đọc báo cùng bạn    3
Thời trang          3
Theo Gương Bác      3
Chuyện pháp đình    3
Thường thức         2
Mẹ & Bé             2
Âm nhạc             2
Gia đình số         2
Mách bạn            2
MeKong Xanh         2
Sàn diễn            2
Thị trường 247      2
Ăn gì               2
Nhịp cầu            2
Hồ sơ               2
 Điện ảnh           1
Tiêu điểm           1
Thể thao vui        1
Môi Trường          1
Kiều bào            1
Nhân vật            1
Name: class, dtype: int64